<h1>Auto ETL Assistant</h1>

This notebook is meant as an entrypoint to use the Auto ETL Assistant. The assistant will write metadata descriptions for input tables and columns using ingested `yml` files structured in the following way:
<br>

```
tables:
        - name: foo
            description: Description of the table
            columns:
            - name: column1
                description: Description of column1
            - name: column2
                description: Description of column2
        - name: bar
            description: Description of another table
            columns:
            - name: columnA
                description: Description of columnA
```

The underlying libraries are included, however, if you would like to process a directory of `yml` metadata and apply them to the associated tables in a single `catalog.schema` defined by the `SOURCE_CATALOG` and `SOURCE_SCHEMA` configuration variables below. Please fill in the global variables in the cell below and place your metadata in the `./metadata` directory in this repository. Alternatively, you can change the path to the metadata files in the global variables using the `PATH_TO_METADATA` config variable below.

Use the `DESTINATION_CATALOG`, `DESTINATION_SCHEMA`, and `DESTINATION_TABLES` configuration variables to direct the assistant at the desitnation (silver, etc) tables you would like to be the target of the ETL process.

This assistant will use the metadata associated with the source and destination tables to construct (and run, if approved) queries that will transform and load your input tables into your destination tables.

In [ ]:
%pip install pyyaml

In [ ]:
SOURCE_CATALOG = "< your catalog name here >"
SOURCE_SCHEMA = "< your schema name here>"

# Only change the following path if you have a different directory structure
PATH_TO_METADATA = "./metadata"

DESTINATION_CATALOG = "< your destination catalog name here >"
DESTINATION_SCHEMA = "< your destination schema name here >"
DESTINATION_TABLES = ["< your destination table names here >"]

In [ ]:
from utils.register_sql_functions import register_describe_table_function
register_describe_table_function(catalog=SOURCE_CATALOG)

In [ ]:
from utils.metadata_processor import process_metadata
process_metadata(PATH_TO_METADATA, SOURCE_CATALOG, SOURCE_SCHEMA)

In [ ]:
# Run this cell to register the LLM ETL Agent
result = dbutils.notebook.run("./agents/ua-genie-agents-etl-assist", 500, {"source_catalog": SOURCE_CATALOG, 
                                                                          "source_schema": SOURCE_SCHEMA, 
                                                                          "destination_catalog": DESTINATION_CATALOG, 
                                                                          "destination_schema": DESTINATION_SCHEMA, 
                                                                          "destination_tables": DESTINATION_TABLES})